# Calculation of returns for various mortgage options

## NPV calculations

Assume you have a payment at time $i$ represented by $p_i$. Assume that an investment of money right now can get you a rate $r$ per period. Then, the present value of that payment to you is $\frac{p_i}{(1+r)^i}$.

Let ${\bf p}$ represent the list of payments $p_i$ over time.

Then, the net present value of all such payments is
\begin{equation} 
{\rm NPV}({\bf p}, i, r) = \sum_{i=0}^{n-1} \frac{p_i}{(1+r)^i}
\end{equation}

### Example

Assume you have a mortgage loan of $300,000 at 2.75%. That results in a monthly payment as follows.

In [1]:
import numpy_financial

mortgage_rate_per_month = 2.75/(12*100)
num_periods = 30*12
present_value = 300000

val = numpy_financial.pmt(mortgage_rate_per_month, num_periods, present_value)
monthly_payment = (-1)*val.item()

print("Monthly payment is $%.2f" % (monthly_payment))

Monthly payment is $1224.72


### Relation between mortgage payments and NPV

If an alternative investment pays only the mortgage rate that you get from your bank, then the NPV of your mortgage payments along with the money you get at the beginning is 0. This is shown by the code below.

In [2]:
payments_list = [present_value]
payments_list.extend([(-1)*monthly_payment]*num_periods)

round(numpy_financial.npv(mortgage_rate_per_month, payments_list)) # This is a number close to 0

0

A positive NPV is desirable. It means that the payments and returns are a reasonable investment. For instance, if the stock market yields a higher rate than the mortgage rate, then the NPV is greater than 0. This is because you can take the mortgage loan, invest it in the stock market, make the mortgage payments from the investment income and still come out positive.

In [3]:
stock_market_rate_per_month = 6/(12*100)
npv = numpy_financial.npv(stock_market_rate_per_month, payments_list)

print("NPV for %d payments with a %d%% stock market return is $%.2f" % (num_periods, stock_market_rate_per_month*12*100, npv))

NPV for 360 payments with a 6% stock market return is $95726.38


In the presence of a strong stock market option, there is a higher NPV in stretching out the payments.

In [4]:
num_periods_long = 35*12

val = numpy_financial.pmt(mortgage_rate_per_month, num_periods_long, present_value)
monthly_payment_long = (-1)*val.item()

payments_list_long = [present_value]
payments_list_long.extend([(-1)*monthly_payment_long]*num_periods_long)
npv = numpy_financial.npv(stock_market_rate_per_month, payments_list_long)

print("NPV for %d payments with a %d%% stock market return is $%.2f" % (num_periods_long, stock_market_rate_per_month*12*100, npv))

NPV for 420 payments with a 6% stock market return is $104783.61


## Partial periods

This may seem obvious but, if you look at the paid-off principal at any point in the mortgage period, the payment comes out the same if you were to get a new loan for the reduced principal. The code below illustrates that.

In [5]:
full_pmts = 30*12
first_half_pmts = 90

val = numpy_financial.pmt(mortgage_rate_per_month, full_pmts, present_value)
full_period_monthly_pmt = (-1)*val.item()

val_list = numpy_financial.ppmt(mortgage_rate_per_month, [i + 1 for i in range(first_half_pmts)], full_pmts, present_value)
ppmt = [(-1)*val.item() for val in val_list]

new_present_value = present_value - sum(ppmt) # Amount is partially paid off
val = numpy_financial.pmt(mortgage_rate_per_month, full_pmts - first_half_pmts, new_present_value)
half_period_monthly_pmt = (-1)*val.item()

print("Principal remaining for partial period loan is $%.2f" % (new_present_value))
print("Full period payment is $%.2f while half period payment is $%.2f" % (full_period_monthly_pmt, half_period_monthly_pmt))

Principal remaining for partial period loan is $246370.53
Full period payment is $1224.72 while half period payment is $1224.72


## Evaluation of scenarios

Let's say that, in scenario 1, you are offered a high mortgage rate with no initial payment. In scenario 2, you are offered a lower rate but with a one-time fee associated. Which is better?

In [6]:
present_value = 280000

s1_rate_per_month = 3.375/(12*100)
s1_num_periods = 270

s1_monthly_payment = numpy_financial.pmt(s1_rate_per_month, s1_num_periods, present_value).item()*(-1)

s1_payments_list = [present_value]
s1_payments_list.extend([(-1)*s1_monthly_payment]*s1_num_periods)
s1_npv = numpy_financial.npv(stock_market_rate_per_month, s1_payments_list)

s2_rate_per_month = 2.75/(12*100) # Lower interest rate than S1...
s2_num_periods = 360
s2_one_time_fee = 6000 # ... but there is a one-time payment that does not exist in S1

s2_monthly_payment = numpy_financial.pmt(s2_rate_per_month, s2_num_periods, present_value).item()*(-1)

s2_payments_list = [present_value - s2_one_time_fee]
s2_payments_list.extend([(-1)*s2_monthly_payment]*s2_num_periods)
s2_npv = numpy_financial.npv(stock_market_rate_per_month, s2_payments_list)

print("Monthly payments are:\nS1: $%.2f\nS2: $%.2f\n" % (s1_monthly_payment, s2_monthly_payment))
print("NPV for S1 at %d%% stock market rate is $%.2f" % (stock_market_rate_per_month*12*100, s1_npv))
print("NPV for S2 at %d%% stock market rate is $%.2f\n" % (stock_market_rate_per_month*12*100, s2_npv))

if (s1_npv >= s2_npv):
    print ("S1 is better by an NPV of $%.2f" % (s1_npv - s2_npv))
else:
    print ("S2 is better by an NPV of $%.2f" % (s2_npv - s1_npv))


Monthly payments are:
S1: $1481.54
S2: $1143.08

NPV for S1 at 6% stock market rate is $60766.05
NPV for S2 at 6% stock market rate is $83344.62

S2 is better by an NPV of $22578.57
